In [44]:
import requests
import pandas as pd
import random
import time
import datetime
from bs4 import BeautifulSoup
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
# Retrieve API given client code
def get_auth_token():
    client_id = "c92bf1c7c40f4500a2b4e2a1faac4997"
    client_secret = "a00809b4c9734ea38fe0d4b05719b62c"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}"
    url = "https://accounts.spotify.com/api/token"

    response = requests.post(url=url, data=data, headers=headers)
    token = response.json()["access_token"]
    print(f"TOKEN {token} GENERATED")
    return {"Authorization": f"Bearer  {token}"}

auth = []
auth.append(get_auth_token())

TOKEN BQCsO87Xvte9ZQ31EXNDBMeYLVdYVyVZHWbNSYABS9RC42J5yKS_9izDT1X3W-4ermRDosCFwazq2ASASRP5mXGuNnVlEFZnxR3L8zeFxfhcGr7LtBw GENERATED


In [128]:
def extract_artist(artist_json: dict):
    id = artist_json["id"]
    name = artist_json["name"]
    genres = artist_json["genres"]
    followers = artist_json["followers"]
    popularity = artist_json["popularity"]
    return {
        "id": id,
        "name": name,
        "genres": genres,
        "followers": followers,
        "popularity": popularity
        }

def get_artist(a_id: str, auth: list):
    artist_url = f"https://api.spotify.com/v1/artists/{a_id}"
    response = requests.get(url=artist_url, headers=auth[-1])

    # check for expired token
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_artist(a_id, auth)
    return response.json()

def get_related_artists(a_id: str, auth: list):
    related_url = f"https://api.spotify.com/v1/artists/{a_id}/related-artists"
    response = requests.get(url=related_url, headers=auth[-1])
    
    # check for expired token
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_related_artists(a_id, auth)

    return response.json()   

def extract_related_artists(related_json: dict, visited: set, sources: list):
    artists = []
    for a in related_json["artists"]:
        if a["id"] not in visited:
            visited.add(a["id"])
            sources.append(a["id"])
            artists.append(extract_artist(a))
    return artists

In [5]:
# initialization
# artist_dataset = pd.DataFrame(columns=["id", "name", "genres", "followers", "popularity"])
# visited_artists = set([]) # set of artists who are already in the dataset
# source_artists = [] # list of artists whose related artists have not been added

# reloading from saved point
artist_dataset = pd.read_csv("artist_dataset-2024-04-09.csv")
visited_artists = set(artist_dataset['id'].to_list())
source_artists = artist_dataset['id'].to_list()[10000:]

In [95]:
# DEPRECATED, JUST FOR STARTING SEARCH

# starting_artists = []
# with open("starting_artists.txt", 'r') as f:
#     starting_artists = f.read().splitlines()

# for a in starting_artists:
#     artist_dataset.loc[len(artist_dataset)] = extract_artist(get_artist(a, auth))
#     visited_artists.add(a)
#     source_artists.append(a)

In [170]:
while len(visited_artists) < 75000 and len(source_artists) > 0:
    time.sleep(.5) # janky rate limiting
    search_id = source_artists.pop(random.randint(0, len(source_artists) - 1))
    related_artists = get_related_artists(search_id, auth)
    new_artists = pd.DataFrame(extract_related_artists(related_artists, visited_artists, source_artists))
    print(f"{len(new_artists)} ADDED")
    artist_dataset = pd.concat([artist_dataset, new_artists], ignore_index=True)

1 ADDED
10 ADDED
4 ADDED
14 ADDED
3 ADDED
1 ADDED
14 ADDED
10 ADDED
6 ADDED
11 ADDED
4 ADDED
2 ADDED
13 ADDED
6 ADDED
0 ADDED
8 ADDED
2 ADDED
10 ADDED
18 ADDED
6 ADDED
5 ADDED
4 ADDED
4 ADDED
6 ADDED
2 ADDED
14 ADDED
1 ADDED
9 ADDED
9 ADDED
0 ADDED
4 ADDED
3 ADDED
3 ADDED
11 ADDED
0 ADDED
10 ADDED
14 ADDED
14 ADDED
5 ADDED
13 ADDED
3 ADDED
5 ADDED
19 ADDED
10 ADDED
2 ADDED
14 ADDED
9 ADDED
4 ADDED
17 ADDED
2 ADDED
10 ADDED
11 ADDED
1 ADDED
5 ADDED
11 ADDED
1 ADDED
1 ADDED
19 ADDED
5 ADDED
11 ADDED
14 ADDED
0 ADDED
9 ADDED
1 ADDED
0 ADDED
4 ADDED
12 ADDED
0 ADDED
4 ADDED
0 ADDED
2 ADDED
12 ADDED
5 ADDED
6 ADDED
0 ADDED
13 ADDED
18 ADDED
5 ADDED
9 ADDED
13 ADDED
12 ADDED
9 ADDED
0 ADDED
6 ADDED
0 ADDED
0 ADDED
1 ADDED
9 ADDED
9 ADDED
11 ADDED
2 ADDED
3 ADDED
11 ADDED
2 ADDED
3 ADDED
1 ADDED
7 ADDED
0 ADDED
4 ADDED
3 ADDED
5 ADDED
4 ADDED
8 ADDED
7 ADDED
3 ADDED
6 ADDED
7 ADDED
11 ADDED
7 ADDED
3 ADDED
11 ADDED
10 ADDED
10 ADDED
2 ADDED
14 ADDED
0 ADDED
3 ADDED
12 ADDED
9 ADDED
10 ADDED
1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
removal_list = []
def remove_artist(a_id):
    removal_list.append(a_id)    

In [12]:
def get_top_tracks(a_id: str, auth: list):
    url = f"https://api.spotify.com/v1/artists/{a_id}/top-tracks"
    response = requests.get(url, headers=auth[-1])
    
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_related_artists(a_id, auth[-1])

    # wait for clearance if request is denied for rate limiting reasons
    if response.status_code == 429:
        sleep = int(response.headers["retry-after"]) + 1
        if sleep > 1800:
            raise BaseException(f"API Limited for too long. Pausing. Try again in {sleep} seconds.")
        print(f"Rate limited. Sleeping for {sleep} seconds.")
        time.sleep(sleep)
        return get_top_tracks(a_id, auth)

    if response.status_code == 404:
        raise LookupError('Track not found')
    
    return response.json()

def extract_top_tracks(tracks_json):
    tracks = []
    for track in tracks_json["tracks"]:
        artists = track["artists"]
        duration = track["duration_ms"]
        explicit = track["explicit"]
        id = track["id"]
        name = track["name"]
        popularity = track["popularity"]
        row = {
            "id": id,
            "name": name,
            "artists": artists,
            "duration": duration,
            "explicit": explicit,
            "popularity": popularity
        }
        tracks.append(row)
    return tracks

In [8]:
song_dataset = pd.DataFrame(columns=['id', 'name', 'artists', 'duration', 'explicit', 'popularity'])

In [10]:
ids = artist_dataset['id'].to_list()
# last_searched = ids.index("3KedxarmBCyFBevnqQHy3P")
last_searched = -1

In [13]:
for a_id in artist_dataset['id'].to_list()[last_searched + 1:]:
    try:
        # time.sleep(.5)
        t = get_top_tracks(a_id, auth)
        new_songs = pd.DataFrame(extract_top_tracks(t))
        print(f"{len(new_songs)} songs ADDED for artist: {a_id}")
        song_dataset = pd.concat([song_dataset, new_songs], ignore_index=True)
    except LookupError as e:
        print(f"No songs found for artist: {a_id}, adding to list.")
        remove_artist(a_id)
    except BaseException as et:
        print(et)
        song_dataset.to_csv("song_dataset.csv")
        break
    

API Limited for too long. Pausing. Try again in 46655 seconds.


In [31]:
# URL of the Wikipedia page
wiki_url = 'https://en.wikipedia.org/wiki/Coachella_Festival_line-ups'

# Fetch the web page content
response = requests.get(wiki_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', class_='wikitable')

table_years = [1999, 1999, 2001, 2002, 2002, 2003, 2003, 2004, 2004, 2005, 2005, 2006, 2006, 2007, 2007, 2007, 2008, 2008, 2008, 2009, 2009, 2009, 2010, 2010, 2010, 2011, 2011, 2011, 2012, 2012, 2012, 2013, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2015, 2016, 2016, 2016, 2017, 2017, 2017, 2018, 2018, 2018, 2019, 2019, 2019, 2020, 2022, 2022, 2022, 2023, 2023, 2023]
performers_data = []

# Iterate through each table and extract its contents
for index, table in enumerate(tables, start=1):
    year = table_years[index - 1]
    if year == 2020:
        continue

    rows = table.find_all('tr')
    header_row = True
    headers = []
    performers = []
    for row in rows:
        cells = row.find_all(['th', 'td'])
        counter = 0
        for cell in cells:
            if header_row:
                headers.append(cell.get_text(strip=True))
            else:
                performers = cell.get_text(strip=False).split("\n")[1:-1]
                performers_data.append({
                    "stage": headers[counter],
                    "year": year,
                    "performers": performers
                    })
                counter += 1
        header_row = False

In [38]:
def remove_parentheses(text):
    pattern = r'\([^)]*\)'
    clean_text = re.sub(pattern, '', text)
    return clean_text.strip()

In [46]:
# data transformation, creating intermediate tables
performers_df = pd.DataFrame(data=performers_data)
singular = performers_df.explode(column="performers")
singular["name"] = singular["performers"].apply(remove_parentheses)
singular = singular.drop(labels="performers", axis=1)

In [49]:
def get_artist_ids(artist_names : list):

    client_credentials_manager = SpotifyClientCredentials(client_id="c92bf1c7c40f4500a2b4e2a1faac4997",
                                                        client_secret='a00809b4c9734ea38fe0d4b05719b62c')
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    artist_dict = {}
    for artist_name in artist_names:
        results = sp.search(q='artist:' + artist_name, type='artist', limit=1)
        
        if results['artists']['items']:
            artist_id = results['artists']['items'][0]['id']
            artist_dict[artist_name] = artist_id
        else:
            artist_dict[artist_name] = None

    artist_df = pd.DataFrame([artist_dict]).T.reset_index()
    artist_df.columns = ['name', 'id']
    return artist_df 

In [50]:
performer_ids = get_artist_ids(singular["name"].to_list())

ValueError: You are trying to merge on object and int64 columns for key 'name'. If you wish to proceed you should use pd.concat

In [83]:
artist_dataset = pd.merge(performer_ids, singular, how="left", on="name")
artist_dataset = artist_dataset.drop_duplicates(subset=["name", "stage", "year"])
artist_dataset = artist_dataset.dropna(subset="id")
artist_dataset = artist_dataset.reset_index().drop(labels='index', axis=1)
artist_dataset

,name,id,stage,year
0,Beck,4obzFoKoKRHIphyHzJ35G3,Main Stage,1999
1,Beck,4obzFoKoKRHIphyHzJ35G3,Gobi Tent,2004
2,Beck,4obzFoKoKRHIphyHzJ35G3,Coachella Stage,2014
3,The Chemical Brothers,1GhPHrq36VKCY3ucVaZCfo,Main Stage,1999
4,The Chemical Brothers,1GhPHrq36VKCY3ucVaZCfo,Insomniac Tent,2001
...,...,...,...,...
2907,Cassian,1ChtRJ3f4rbv4vtz87i6CD,Yuma Tent,2023
2908,TSHA,2kLa7JZu4Ijdz1Gle2khZh,Yuma Tent,2023
2909,LP Giobbi,3oKnyRhYWzNsTiss5n4Z1J,Yuma Tent,2023
2910,Airrica,57sPl5iWgq5t6AscVbMTOW,Yuma Tent,2023


In [84]:
# save data to csv
date = datetime.datetime.today().strftime('%Y-%m-%d')
artist_dataset.to_csv(f"coachella_artists.csv")